In [1]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization
import os 
import pandas as pd
import plotly.graph_objs as go
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [2]:
!pip install opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/amrragababdelaziz/dermnet-and-skin-disease-9-classes')















Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds

Your Kaggle username: rubinie

Your Kaggle Key: ········



100%|█████████████████████████████████████████████████████████████████████████████| 3.51G/3.51G [03:48<00:00, 16.5MB/s]


In [7]:
train ='C:\\Users\\rubin\\Videos\\FINAL REVIEW\\9-Skin-Cancer\\Merged Dataset\\Train'
validation_dir = 'C:\\Users\\rubin\\Videos\\FINAL REVIEW\\9-Skin-Cancer\\Merged Dataset\\test'

In [8]:
import glob
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [9]:
train_samples =get_files(train)
num_classes=len(glob.glob(train+"/*"))
test_samples=get_files(validation_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

9 Classes

26642 Train images

2389 Test images


In [10]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)
img_width,img_height =224,224
input_shape=(img_width,img_height,3)
batch_size =64
train_generator =train_datagen.flow_from_directory(train,
                                target_size=(img_width,img_height),batch_size=batch_size)
test_generator=test_datagen.flow_from_directory(validation_dir,shuffle=True,target_size=(img_width,img_height),batch_size=batch_size)

Found 26642 images belonging to 9 classes.

Found 2389 images belonging to 9 classes.


In [11]:
from tensorflow import keras
from keras import layers
model = keras.Sequential([
    layers.Input((224, 224, 3)),
    
    layers.Conv2D(16, 3, activation="relu"),
    layers.BatchNormalization(), # Regularization
    layers.MaxPooling2D(), 
    
    layers.Conv2D(32, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    
    layers.Dense(64, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.2), # Regularization
    layers.Dense(9, activation="softmax"),
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
history = model.fit(train_generator,validation_data=test_generator,epochs=70,shuffle=True)

Epoch 1/70

417/417 [==============================] - 625s 1s/step - loss: 1.6646 - accuracy: 0.4160 - val_loss: 2.6614 - val_accuracy: 0.1444

Epoch 2/70

417/417 [==============================] - 615s 1s/step - loss: 1.4247 - accuracy: 0.4920 - val_loss: 1.8593 - val_accuracy: 0.3349

Epoch 3/70

417/417 [==============================] - 611s 1s/step - loss: 1.3237 - accuracy: 0.5206 - val_loss: 1.8801 - val_accuracy: 0.3474

Epoch 4/70

417/417 [==============================] - 608s 1s/step - loss: 1.2559 - accuracy: 0.5528 - val_loss: 2.1767 - val_accuracy: 0.2537

Epoch 5/70

417/417 [==============================] - 605s 1s/step - loss: 1.1965 - accuracy: 0.5712 - val_loss: 1.6724 - val_accuracy: 0.4165

Epoch 6/70

417/417 [==============================] - 607s 1s/step - loss: 1.1435 - accuracy: 0.5941 - val_loss: 1.6150 - val_accuracy: 0.4391

Epoch 7/70

417/417 [==============================] - 593s 1s/step - loss: 1.0811 - accuracy: 0.6192 - val_loss: 1.8912 - val_acc

In [41]:
model.save('CNN_model.h5')

In [42]:
img='C:\\Users\\rubin\\Videos\\FINAL REVIEW\\9-Skin-Cancer\\Merged Dataset\\Train\\Psoriasis pictures Lichen Planus and related diseases\\0_36.jpg'

In [43]:
def predict_disease(test_image):
  img = cv2.imread(test_image)
  img = img / 255.0
  img = cv2.resize(img, (224, 224))
  img = img.reshape(1,224,224,3)
  prediction = model.predict(img)
  pred_class = np.argmax(prediction, axis = -1)
  return pred_class

In [44]:
pred=predict_disease(img)
d=pred[0]

1/1 [==============================] - 0s 26ms/step


In [45]:
skin_conditions = [    "Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions",    "Atopic Dermatitis Photos",    "Eczema Photos",    "Melanoma Skin Cancer Nevi and Moles",    "Nail Fungus and other Nail Disease",    "Psoriasis pictures Lichen Planus and related diseases",    "Seborrheic Keratoses and other Benign Tumors",    "Tinea Ringworm Candidiasis and other Fungal Infections",    "Warts Molluscum and other Viral Infections"]


In [46]:
skin_conditions[d]

'Psoriasis pictures Lichen Planus and related diseases'